In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/IEOR 142 NBA Final Project ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nba_playoff_contenders = pd.read_excel("NBA_playoff_contenders.xlsx")
nba_playoff_contenders

,Team,Year,Seed,Champion Share Score,Playoff Experience,Age,W,L,PW,PL,...,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,Opp eFG%,Opp TOV%,DRB%,Opp FT/FGA
0,Los Angeles Lakers*,2000,1,1.000000,672,29.2,67,15,64,18,...,0.153,0.525,0.484,12.7,30.6,0.241,0.443,13.4,73.1,0.222
1,Portland Trail Blazers*,2000,2,0.666667,567,29.6,59,23,59,23,...,0.175,0.546,0.501,14.5,30.3,0.240,0.461,13.8,72.4,0.217
2,San Antonio Spurs*,2000,4,0.066667,623,30.9,53,29,58,24,...,0.138,0.535,0.488,14.3,27.8,0.258,0.451,13.5,73.0,0.188
3,Phoenix Suns*,2000,5,0.266667,396,28.6,53,29,56,26,...,0.184,0.532,0.491,15.2,29.3,0.217,0.454,15.7,70.5,0.245
4,Utah Jazz*,2000,3,0.266667,702,31.5,55,27,54,28,...,0.134,0.540,0.490,14.3,29.5,0.260,0.477,15.0,73.2,0.256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,Toronto Raptors*,2022,5,0.125000,198,24.8,48,34,47,35,...,0.375,0.543,0.510,11.0,28.4,0.177,0.535,14.4,75.6,0.199
508,Atlanta Hawks*,2022,8,0.062500,271,26.1,43,39,45,37,...,0.390,0.581,0.543,10.8,23.0,0.205,0.543,11.5,76.9,0.177
509,Brooklyn Nets*,2022,7,0.000000,507,29.1,44,38,43,39,...,0.359,0.576,0.540,12.5,23.9,0.198,0.521,11.7,75.1,0.201
510,Chicago Bulls*,2022,6,0.062500,204,26.3,46,36,40,42,...,0.332,0.579,0.541,11.8,20.4,0.201,0.541,11.9,78.3,0.199


# Approach 1: Multiple Linear Regression
---

In [ ]:
from sklearn.model_selection import train_test_split

# Dropping the team names, since they won't be helpful in our model. 
nba_playoff_contenders.drop(["Team"], axis=1, inplace=True) 

X = nba_playoff_contenders.drop(["Champion Share Score"], axis=1) # Removing the feature to predict
y = nba_playoff_contenders["Champion Share Score"] # The variable we are interested in

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
import statsmodels.api as sm

X_train = sm.add_constant(X_train)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.566
Model:                              OLS   Adj. R-squared:                  0.535
Method:                   Least Squares   F-statistic:                     18.11
Date:                  Tue, 09 May 2023   Prob (F-statistic):           8.11e-47
Time:                          20:51:34   Log-Likelihood:                 75.465
No. Observations:                   358   AIC:                            -100.9
Df Residuals:                       333   BIC:                            -3.917
Df Model:                            24                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

### Feature Selection via Variance Inflation Factors and P-values

In [ ]:
# calculate Variance Inflation Factor for each explanatory variable
from statsmodels.stats.outliers_influence import variance_inflation_factor

def VIF(df):
    vif = [variance_inflation_factor(df, i) for i in range(1, len(df.columns))]
    return pd.Series(vif, index=df.columns.drop("const"))

In [ ]:
VIF(X_train)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Year                  1.048923e+01
Seed                  4.386161e+00
Playoff Experience    2.772043e+00
Age                   2.816543e+00
W                              inf
L                              inf
PW                             inf
PL                             inf
MOV                   3.087714e+05
SOS                   6.121713e+03
SRS                   3.092867e+05
ORtg                           inf
DRtg                           inf
NRtg                           inf
Pace                  4.653074e+00
FTr                   3.664289e+02
3PAr                  1.066085e+01
TS%                   2.303529e+03
eFG%                  2.272515e+03
TOV%                  8.107083e+01
ORB%                  1.160606e+02
FT/FGA                7.306490e+02
Opp eFG%              1.949745e+02
Opp TOV%              4.089161e+01
DRB%                  3.202213e+01
Opp FT/FGA            1.243079e+01
dtype: float64

In [ ]:
# remove PW because of its high VIF
X_train.drop(["PW"], axis=1, inplace=True)
X_test.drop(["PW"], axis=1, inplace=True)
VIF(X_train)

/usr/local/lib/python3.10/dist-packages/statsmodels/stats/outliers_influence.py:195: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Year                  1.048923e+01
Seed                  4.386161e+00
Playoff Experience    2.772043e+00
Age                   2.816543e+00
W                     1.950261e+01
L                     2.272333e+01
PL                    1.021833e+02
MOV                   3.087714e+05
SOS                   6.121713e+03
SRS                   3.092867e+05
ORtg                           inf
DRtg                           inf
NRtg                           inf
Pace                  4.653074e+00
FTr                   3.664289e+02
3PAr                  1.066085e+01
TS%                   2.303529e+03
eFG%                  2.272515e+03
TOV%                  8.107083e+01
ORB%                  1.160606e+02
FT/FGA                7.306490e+02
Opp eFG%              1.949745e+02
Opp TOV%              4.089161e+01
DRB%                  3.202213e+01
Opp FT/FGA            1.243079e+01
dtype: float64

In [ ]:
# remove ORtg because of its high VIF
X_train.drop(["ORtg"], axis=1, inplace=True)
X_test.drop(["ORtg"], axis=1, inplace=True)
VIF(X_train)

Year                      10.489227
Seed                       4.386161
Playoff Experience         2.772043
Age                        2.816543
W                         19.502613
L                         22.723328
PL                       102.183334
MOV                   308771.435310
SOS                     6121.712652
SRS                   309286.665932
DRtg                     987.589387
NRtg                     798.122607
Pace                       4.653074
FTr                      366.428921
3PAr                      10.660848
TS%                     2303.529020
eFG%                    2272.514896
TOV%                      81.070835
ORB%                     116.060643
FT/FGA                   730.648958
Opp eFG%                 194.974535
Opp TOV%                  40.891607
DRB%                      32.022127
Opp FT/FGA                12.430790
dtype: float64

In [ ]:
# remove SRS because of its high VIF
X_train.drop(["SRS"], axis=1, inplace=True)
X_test.drop(["SRS"], axis=1, inplace=True)
VIF(X_train)

Year                    10.469619
Seed                     4.371829
Playoff Experience       2.727616
Age                      2.802218
W                       19.499628
L                       22.713333
PL                     102.143506
MOV                    511.586903
SOS                      1.361443
DRtg                   987.387180
NRtg                   798.028481
Pace                     4.650177
FTr                    366.018217
3PAr                    10.651288
TS%                   2300.395664
eFG%                  2268.703165
TOV%                    81.059198
ORB%                   116.017081
FT/FGA                 729.636849
Opp eFG%               194.959985
Opp TOV%                40.847023
DRB%                    32.021727
Opp FT/FGA              12.399295
dtype: float64

In [ ]:
# remove TS% because of its high VIF
X_train.drop(["TS%"], axis=1, inplace=True)
X_test.drop(["TS%"], axis=1, inplace=True)
VIF(X_train)

Year                    9.856960
Seed                    4.365545
Playoff Experience      2.697696
Age                     2.798643
W                      18.939525
L                      22.312584
PL                    100.598606
MOV                   510.965762
SOS                     1.348281
DRtg                  897.670422
NRtg                  777.653813
Pace                    4.648351
FTr                   115.379501
3PAr                   10.401456
eFG%                  464.184420
TOV%                   75.310569
ORB%                  107.743945
FT/FGA                198.163426
Opp eFG%              183.121474
Opp TOV%               37.911692
DRB%                   28.718079
Opp FT/FGA             11.415628
dtype: float64

In [ ]:
# remove DRtg because of its high VIF
X_train.drop(["DRtg"], axis=1, inplace=True)
X_test.drop(["DRtg"], axis=1, inplace=True)
VIF(X_train)

Year                    9.606460
Seed                    4.287223
Playoff Experience      2.693686
Age                     2.794296
W                      18.809920
L                      22.297974
PL                    100.389838
MOV                   510.582420
SOS                     1.325878
NRtg                  608.413537
Pace                    4.641843
FTr                    23.651692
3PAr                    6.288523
eFG%                   70.964561
TOV%                   13.116403
ORB%                   17.843420
FT/FGA                 33.400055
Opp eFG%               69.331392
Opp TOV%               15.680199
DRB%                   11.464252
Opp FT/FGA              5.199733
dtype: float64

In [ ]:
# remove NRtg because of its high VIF
X_train.drop(["NRtg"], axis=1, inplace=True)
X_test.drop(["NRtg"], axis=1, inplace=True)
VIF(X_train)

Year                   9.561249
Seed                   4.276797
Playoff Experience     2.693619
Age                    2.793563
W                     17.070588
L                     20.415877
PL                    87.785207
MOV                   85.617932
SOS                    1.311774
Pace                   3.269355
FTr                   23.627496
3PAr                   6.253828
eFG%                  68.661209
TOV%                  12.732531
ORB%                  17.420376
FT/FGA                33.037244
Opp eFG%              68.027333
Opp TOV%              15.435578
DRB%                  11.337101
Opp FT/FGA             5.060548
dtype: float64

In [ ]:
# remove PL because of its high VIF
X_train.drop(["PL"], axis=1, inplace=True)
X_test.drop(["PL"], axis=1, inplace=True)
VIF(X_train)

Year                   9.277188
Seed                   4.268507
Playoff Experience     2.670815
Age                    2.792699
W                      3.636160
L                      5.496233
MOV                   38.179677
SOS                    1.228075
Pace                   3.046301
FTr                   23.548203
3PAr                   6.202343
eFG%                  67.233331
TOV%                  12.231593
ORB%                  17.145123
FT/FGA                32.766414
Opp eFG%              66.145994
Opp TOV%              14.781328
DRB%                  10.876285
Opp FT/FGA             4.874180
dtype: float64

In [ ]:
# remove Opp eFG% because of its high VIF
X_train.drop(["Opp eFG%"], axis=1, inplace=True)
X_test.drop(["Opp eFG%"], axis=1, inplace=True)
VIF(X_train)

Year                   9.221945
Seed                   4.265485
Playoff Experience     2.642059
Age                    2.786705
W                      3.592119
L                      5.485883
MOV                    7.725972
SOS                    1.222414
Pace                   2.834072
FTr                   13.158942
3PAr                   5.292725
eFG%                   6.565900
TOV%                   1.818809
ORB%                   3.286597
FT/FGA                11.785013
Opp TOV%               2.345564
DRB%                   4.329240
Opp FT/FGA             2.035313
dtype: float64

In [ ]:
# remove FTr because of its high VIF
X_train.drop(["FTr"], axis=1, inplace=True)
X_test.drop(["FTr"], axis=1, inplace=True)
VIF(X_train)

Year                  8.884078
Seed                  4.264688
Playoff Experience    2.621411
Age                   2.767563
W                     3.591535
L                     5.436579
MOV                   7.673895
SOS                   1.215727
Pace                  2.827711
3PAr                  5.229926
eFG%                  6.558994
TOV%                  1.787471
ORB%                  3.095048
FT/FGA                1.771240
Opp TOV%              2.338202
DRB%                  4.268906
Opp FT/FGA            2.030275
dtype: float64

In [ ]:
# remove Year because of its high VIF
X_train.drop(["Year"], axis=1, inplace=True)
X_test.drop(["Year"], axis=1, inplace=True)
VIF(X_train)

Seed                  4.255770
Playoff Experience    2.511334
Age                   2.463787
W                     3.584743
L                     5.425288
MOV                   7.152927
SOS                   1.214143
Pace                  2.819134
3PAr                  4.820454
eFG%                  5.679060
TOV%                  1.668118
ORB%                  3.088071
FT/FGA                1.771143
Opp TOV%              2.197296
DRB%                  2.521315
Opp FT/FGA            1.914348
dtype: float64

In [ ]:
# remove MOV because of its high VIF
X_train.drop(["MOV"], axis=1, inplace=True)
X_test.drop(["MOV"], axis=1, inplace=True)
VIF(X_train)

Seed                  4.255380
Playoff Experience    2.510313
Age                   2.457376
W                     2.462140
L                     2.857290
SOS                   1.189780
Pace                  2.818686
3PAr                  4.819932
eFG%                  5.409616
TOV%                  1.667645
ORB%                  2.926332
FT/FGA                1.766570
Opp TOV%              2.153343
DRB%                  2.512699
Opp FT/FGA            1.868975
dtype: float64

In [ ]:
# remove eFG% because of its high VIF
X_train.drop(["eFG%"], axis=1, inplace=True)
X_test.drop(["eFG%"], axis=1, inplace=True)
VIF(X_train)

Seed                  4.219299
Playoff Experience    2.398240
Age                   2.401320
W                     2.177822
L                     2.645783
SOS                   1.184140
Pace                  2.640581
3PAr                  3.541710
TOV%                  1.634271
ORB%                  2.522377
FT/FGA                1.765144
Opp TOV%              1.893978
DRB%                  2.427647
Opp FT/FGA            1.813443
dtype: float64

In [ ]:
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.556
Model:                              OLS   Adj. R-squared:                  0.538
Method:                   Least Squares   F-statistic:                     30.67
Date:                  Tue, 09 May 2023   Prob (F-statistic):           4.19e-52
Time:                          20:51:35   Log-Likelihood:                 71.254
No. Observations:                   358   AIC:                            -112.5
Df Residuals:                       343   BIC:                            -54.30
Df Model:                            14                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (3PAr)
X_train.drop(["3PAr"], axis=1, inplace=True)
X_test.drop(["3PAr"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.555
Model:                              OLS   Adj. R-squared:                  0.539
Method:                   Least Squares   F-statistic:                     33.07
Date:                  Tue, 09 May 2023   Prob (F-statistic):           8.40e-53
Time:                          20:51:35   Log-Likelihood:                 71.078
No. Observations:                   358   AIC:                            -114.2
Df Residuals:                       344   BIC:                            -59.83
Df Model:                            13                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (DRB%)
X_train.drop(["DRB%"], axis=1, inplace=True)
X_test.drop(["DRB%"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.555
Model:                              OLS   Adj. R-squared:                  0.539
Method:                   Least Squares   F-statistic:                     35.81
Date:                  Tue, 09 May 2023   Prob (F-statistic):           1.85e-53
Time:                          20:51:35   Log-Likelihood:                 70.756
No. Observations:                   358   AIC:                            -115.5
Df Residuals:                       345   BIC:                            -65.06
Df Model:                            12                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (SOS)
X_train.drop(["SOS"], axis=1, inplace=True)
X_test.drop(["SOS"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.553
Model:                              OLS   Adj. R-squared:                  0.539
Method:                   Least Squares   F-statistic:                     38.91
Date:                  Tue, 09 May 2023   Prob (F-statistic):           5.53e-54
Time:                          20:51:35   Log-Likelihood:                 70.069
No. Observations:                   358   AIC:                            -116.1
Df Residuals:                       346   BIC:                            -69.57
Df Model:                            11                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (W)
X_train.drop(["W"], axis=1, inplace=True)
X_test.drop(["W"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.551
Model:                              OLS   Adj. R-squared:                  0.538
Method:                   Least Squares   F-statistic:                     42.62
Date:                  Tue, 09 May 2023   Prob (F-statistic):           1.61e-54
Time:                          20:51:35   Log-Likelihood:                 69.363
No. Observations:                   358   AIC:                            -116.7
Df Residuals:                       347   BIC:                            -74.04
Df Model:                            10                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (FT/FGA)
X_train.drop(["FT/FGA"], axis=1, inplace=True)
X_test.drop(["FT/FGA"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.549
Model:                              OLS   Adj. R-squared:                  0.537
Method:                   Least Squares   F-statistic:                     47.05
Date:                  Tue, 09 May 2023   Prob (F-statistic):           5.48e-55
Time:                          20:51:35   Log-Likelihood:                 68.437
No. Observations:                   358   AIC:                            -116.9
Df Residuals:                       348   BIC:                            -78.07
Df Model:                             9                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (TOV%)
X_train.drop(["TOV%"], axis=1, inplace=True)
X_test.drop(["TOV%"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.547
Model:                              OLS   Adj. R-squared:                  0.537
Method:                   Least Squares   F-statistic:                     52.74
Date:                  Tue, 09 May 2023   Prob (F-statistic):           1.33e-55
Time:                          20:51:35   Log-Likelihood:                 67.808
No. Observations:                   358   AIC:                            -117.6
Df Residuals:                       349   BIC:                            -82.69
Df Model:                             8                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (ORB%)
X_train.drop(["ORB%"], axis=1, inplace=True)
X_test.drop(["ORB%"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.545
Model:                              OLS   Adj. R-squared:                  0.535
Method:                   Least Squares   F-statistic:                     59.78
Date:                  Tue, 09 May 2023   Prob (F-statistic):           4.66e-56
Time:                          20:51:35   Log-Likelihood:                 66.730
No. Observations:                   358   AIC:                            -117.5
Df Residuals:                       350   BIC:                            -86.42
Df Model:                             7                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
#remove variables with high p-value >.05 (Age)
X_train.drop(["Age"], axis=1, inplace=True)
X_test.drop(["Age"], axis=1, inplace=True)
MLR = sm.OLS(y_train, X_train).fit()
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.542
Model:                              OLS   Adj. R-squared:                  0.534
Method:                   Least Squares   F-statistic:                     69.12
Date:                  Tue, 09 May 2023   Prob (F-statistic):           1.63e-56
Time:                          20:51:35   Log-Likelihood:                 65.581
No. Observations:                   358   AIC:                            -117.2
Df Residuals:                       351   BIC:                            -90.00
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

### Final Model and Metrics


In [ ]:
print(MLR.summary())

                             OLS Regression Results                             
Dep. Variable:     Champion Share Score   R-squared:                       0.542
Model:                              OLS   Adj. R-squared:                  0.534
Method:                   Least Squares   F-statistic:                     69.12
Date:                  Tue, 09 May 2023   Prob (F-statistic):           1.63e-56
Time:                          20:52:29   Log-Likelihood:                 65.581
No. Observations:                   358   AIC:                            -117.2
Df Residuals:                       351   BIC:                            -90.00
Df Model:                             6                                         
Covariance Type:              nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const           

In [ ]:
VIF(X_train)

Seed                  2.402812
Playoff Experience    1.219610
L                     2.322713
Pace                  1.318981
Opp TOV%              1.379894
Opp FT/FGA            1.525917
dtype: float64

In [ ]:
X_test = sm.add_constant(X_test)
y_pred = MLR.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("\nTraining (in-sample) R-squared:", r2_score(y_train, MLR.predict(X_train)))
print("Testing (out-of-sample) R-squared:", r2_score(y_test, y_pred))

Mean Squared Error: 0.047952606705470774

Training (in-sample) R-squared: 0.5416225693844531
Testing (out-of-sample) R-squared: 0.42554189426146416
